In [7]:
# ============================================================================
# JokeBloke: Interactive Voice-Controlled Comedy Assistant
# ============================================================================
# This notebook implements an AI-powered joke generation system with:
# - Voice input/output for natural conversation
# - Multiple comedy personality profiles
# - Contextual memory tracking
# - User feedback learning (UCB algorithm)
# ============================================================================

# Suppress deprecation warnings from pygame and pkg_resources
import warnings
warnings.filterwarnings('ignore', category=UserWarning, message='.*pkg_resources.*')
warnings.filterwarnings('ignore', category=DeprecationWarning, module='pygame')
warnings.filterwarnings('ignore', category=UserWarning, module='pygame.pkgdata')

# Standard Library Imports
import os
import random
import threading
import wave
import subprocess
import xml.etree.ElementTree as ET
from abc import ABC, abstractmethod
from collections import defaultdict
from io import BytesIO
from pathlib import Path
from typing import Literal
import tempfile

# Jupyter and Widget Libraries
import ipywidgets as widgets
from ipywidgets import Output
import ipyaudioworklet as ipyaudio
from IPython.display import display, clear_output

# Data Science and Visualization
import matplotlib.pyplot as plt
import numpy as np

# Audio and Speech Processing
from gtts import gTTS  # Text-to-speech

# Suppress pygame import warnings with context manager
with warnings.catch_warnings():
    warnings.simplefilter("ignore", UserWarning)
    import pygame  # Audio playback

import speech_recognition as sr  # Speech-to-text

# Natural Language Processing
import spacy  # For linguistic feature extraction

# Google Gemini API
from google import genai
from google.genai import types
;

''

In [ ]:
# ============================================================================
# Multi-Language Configuration
# ============================================================================
# JokeBloke ondersteunt meerdere talen / JokeBloke supports multiple languages
# Beschikbare talen / Available languages: 'nl' (Nederlands), 'de' (Deutsch)
# ============================================================================

# Configureer de gewenste taal hier / Configure desired language here
LANGUAGE = 'en-gb'  # Opties / Optionen / Options / Opzioni: 'nl', 'de', 'en-gb', 'it'

# Taalconfiguratie / Language configuration
LANGUAGE_CONFIG = {
    'nl': {
        'name': 'Nederlands',
        'spacy_model': 'nl_core_news_sm',
        'speech_recognition': 'nl-NL',
        'tts_lang': 'nl',
        'ui': {
            'title': 'JokeBloke',
            'instruction': 'Druk op `boot recorder` om hem wakker te maken. Druk vervolgens op `record` om tegen hem te praten. Als je klaar bent met praten, druk dan op `stop` en wacht geduldig op zijn reactie.',
            'like_question': 'Vind je dit leuk?',
            'loading': 'Bedenk iets grappigs...',
            'recorder_ready': 'Recorder klaar.\n',
            'transcription': 'Transcriptie:',
            'could_not_understand': 'Kon audio niet verstaan',
            'api_error': 'API fout:',
            'personality': 'Persoonlijkheid:',
            'response': 'Reactie:',
            'thanks': 'Dank je!',
            'you_liked': 'Je vond dit leuk!',
            'tough_crowd': 'Wow, lastig publiek!',
            'you_disliked': 'Je vond dit niet leuk!',
        },
        'loading_messages': [
            "Even geduld, ik werk eraan...",
            "Komedie kost tijd, anders dan het geduld van mijn ex...",
            "Nog aan het denken... in tegenstelling tot mijn comedycarrière gaat dit ergens heen",
            "Een momentje, genie aan het werk... of op z'n minst lichte entertainment",
            "Humor aan het laden... even geduld alstublieft",
            "Mijn schrijvers staken weer...",
            "Buffer humor... heb je geprobeerd me uit en aan te zetten?",
            "Raadpleeg mijn innerlijke clown...",
            "De clou zit vast in het verkeer...",
            "Bijna klaar... comedy goud groeit niet op bomen",
            "Even volhouden, ik ben grappiger dan deze pauze doet vermoeden",
            "Bezig met verwerken... deze grap kan maar beter goed zijn",
            "Mijn humorbot heeft even nodig...",
            "Grap aan het laden... in tegenstelling tot mijn liefdeleven komt dit wel af",
            "Ik stel niet uit, ik bouw spanning op...",
            "Rome is niet in één dag gebouwd, comedy goud ook niet",
        ]
    },
    'de': {
        'name': 'Deutsch',
        'spacy_model': 'de_core_news_sm',
        'speech_recognition': 'de-DE',
        'tts_lang': 'de',
        'ui': {
            'title': 'JokeBloke',
            'instruction': 'Drücken Sie `boot recorder`, um ihn aufzuwecken. Drücken Sie dann `record`, um mit ihm zu sprechen. Wenn Sie fertig sind, drücken Sie `stop` und warten Sie geduldig auf seine Antwort.',
            'like_question': 'Gefällt dir das?',
            'loading': 'Denke mir etwas Lustiges aus...',
            'recorder_ready': 'Recorder bereit.\n',
            'transcription': 'Transkription:',
            'could_not_understand': 'Konnte Audio nicht verstehen',
            'api_error': 'API-Fehler:',
            'personality': 'Persönlichkeit:',
            'response': 'Antwort:',
            'thanks': 'Danke!',
            'you_liked': 'Das gefiel dir!',
            'tough_crowd': 'Wow, hartes Publikum!',
            'you_disliked': 'Das gefiel dir nicht!',
        },
        'loading_messages': [
            "Einen Moment, ich arbeite daran...",
            "Comedy braucht Zeit, im Gegensatz zur Geduld meiner Ex...",
            "Denke noch nach... im Gegensatz zu meiner Comedy-Karriere führt das irgendwohin",
            "Einen Augenblick, Genie bei der Arbeit... oder zumindest leichte Unterhaltung",
            "Humor wird geladen... bitte warten",
            "Meine Autoren streiken wieder...",
            "Buffere Humor... haben Sie versucht, mich aus- und wieder einzuschalten?",
            "Konsultiere meinen inneren Clown...",
            "Die Pointe steckt im Verkehr fest...",
            "Fast fertig... Comedy-Gold gräbt sich nicht selbst",
            "Halten Sie durch, ich bin lustiger als diese Pause vermuten lässt",
            "Verarbeite... dieser Witz sollte besser gut sein",
            "Mein Humormodul braucht einen Moment...",
            "Witz wird geladen... im Gegensatz zu meinem Liebesleben wird das fertig",
            "Ich zögere nicht, ich baue Spannung auf...",
            "Rom wurde nicht an einem Tag erbaut, Comedy-Gold auch nicht",
        ]
    }
    ,
    'en-gb': {
        'name': 'English (UK)',
        'spacy_model': 'en_core_web_sm',
        'speech_recognition': 'en-GB',
        'tts_lang': 'en',
        'tts_tld': 'co.uk',  # British accent
        'ui': {
            'title': 'JokeBloke',
            'instruction': 'Press `boot recorder` to wake him up. Then press `record` to talk to him. When you\'re done talking, press `stop` and wait patiently for his response.',
            'like_question': 'Do you like this?',
            'loading': 'Thinking of something funny...',
            'recorder_ready': 'Recorder ready.\n',
            'transcription': 'Transcription:',
            'could_not_understand': 'Could not understand audio',
            'api_error': 'API error:',
            'personality': 'Personality:',
            'response': 'Response:',
            'thanks': 'Cheers!',
            'you_liked': 'You liked this!',
            'tough_crowd': 'Blimey, tough crowd!',
            'you_disliked': 'You disliked this!',
        },
        'loading_messages': [
            "Hold on, I'm working on it...",
            "Comedy takes time, unlike my ex's patience...",
            "Still thinking... unlike my comedy career, this is going somewhere",
            "One moment, genius at work... or at least mild amusement",
            "Loading wit... please stand by",
            "My writers are on strike again...",
            "Buffering humour... have you tried turning me off and on again?",
            "Consulting my inner clown...",
            "The punchline is stuck in traffic...",
            "Almost there... comedy gold doesn't mine itself",
            "Hang tight, I'm funnier than this pause suggests",
            "Processing... this joke better be worth it",
            "My funny bone needs a moment...",
            "Joke loading... unlike my love life, this will complete",
            "I'm not stalling, I'm building suspense...",
            "Rome wasn't built in a day, and neither is comedy gold",
        ]
    },
    'it': {
        'name': 'Italiano',
        'spacy_model': 'it_core_news_sm',
        'speech_recognition': 'it-IT',
        'tts_lang': 'it',
        'ui': {
            'title': 'JokeBloke',
            'instruction': 'Premi `boot recorder` per svegliarlo. Poi premi `record` per parlargli. Quando hai finito di parlare, premi `stop` e aspetta pazientemente la sua risposta.',
            'like_question': 'Ti piace?',
            'loading': 'Sto pensando qualcosa di divertente...',
            'recorder_ready': 'Registratore pronto.\n',
            'transcription': 'Trascrizione:',
            'could_not_understand': 'Impossibile comprendere l\'audio',
            'api_error': 'Errore API:',
            'personality': 'Personalità:',
            'response': 'Risposta:',
            'thanks': 'Grazie!',
            'you_liked': 'Ti è piaciuto!',
            'tough_crowd': 'Mamma mia, che pubblico difficile!',
            'you_disliked': 'Non ti è piaciuto!',
        },
        'loading_messages': [
            "Aspetta, ci sto lavorando...",
            "La commedia richiede tempo, a differenza della pazienza della mia ex...",
            "Sto ancora pensando... a differenza della mia carriera comica, questo porterà da qualche parte",
            "Un momento, genio al lavoro... o almeno divertimento leggero",
            "Caricamento umorismo... attendere prego",
            "I miei autori sono di nuovo in sciopero...",
            "Buffering comicità... hai provato a spegnermi e riaccendermi?",
            "Consulto il mio clown interiore...",
            "La battuta finale è bloccata nel traffico...",
            "Quasi fatto... l'oro comico non si estrae da solo",
            "Aspetta, sono più divertente di quanto questa pausa suggerisca",
            "Elaborazione... questa battuta farà meglio essere buona",
            "Il mio senso dell'umorismo ha bisogno di un momento...",
            "Caricamento battuta... a differenza della mia vita amorosa, questo si completerà",
            "Non sto procrastinando, sto creando suspense...",
            "Roma non fu costruita in un giorno, e nemmeno l'oro comico",
        ]
    }
}

# Haal huidige taalconfiguratie op / Get current language configuration
current_lang = LANGUAGE_CONFIG[LANGUAGE]
print(f"🌍 Taal / Sprache: {current_lang['name']}")
print(f"📚 spaCy model: {current_lang['spacy_model']}")
print(f"🎤 Speech recognition: {current_lang['speech_recognition']}")
print(f"🔊 TTS language: {current_lang['tts_lang']}")

In [ ]:
# ============================================================================
# Agent Classes: Core Comedy Generation and Memory Systems
# ============================================================================

class LLMJokerAgent(ABC):
    """
    Abstract base class for LLM-powered joke generation agents.
    
    This class provides a framework for generating jokes using Large Language Models
    with different personalities. It handles profile loading, response generation,
    and conversation memory management.
    
    Attributes:
        MEMORY_SYSTEM_INSTRUCTION (str): System prompt for memory summarization
        prompts_dir (Path): Directory containing personality profile markdown files
        memory (str): Conversation summary maintained across interactions
        output_format (str): XML format specification for joke responses
        profiles (dict): Dictionary mapping profile names to their prompt content
        profile_names (list): List of available personality profile names
    """

    MEMORY_SYSTEM_INSTRUCTION = (
        "You are an advanced agent memory manager that keeps track of conversation "
        "content by maintaining a SHORT summary. You transform the old summary, "
        "with the new user message and agent response to the new summary."
    )

    def __init__(self, prompts_dir: Path):
        """
        Initialize the joker agent and load personality profiles.
        
        Args:
            prompts_dir: Path to directory containing personality .md files
                        and output-format.md specification
        """
        self.prompts_dir = Path(prompts_dir)
        self.memory = ""

        # Load output format specification
        output_format_path = self.prompts_dir / "output-format.md"
        with open(output_format_path, 'r') as f:
            self.output_format = f.read()

        # Load all personality profiles from markdown files
        self.profiles = {}
        for file in self.prompts_dir.iterdir():
            if file.suffix == '.md' and 'output' not in file.name:
                with open(file, 'r') as f:
                    self.profiles[file.stem] = f.read()

        self.profile_names = list(self.profiles.keys())

    @abstractmethod
    def _call_llm(self, system_instruction: str, user_content: str) -> str | None:
        """
        Abstract method for LLM invocation. Subclasses implement this.
        
        Args:
            system_instruction: The system prompt defining agent behavior
            user_content: The user's input message
            
        Returns:
            The LLM's text response, or None if the call fails
        """
        pass

    def get_random_profile(self):
        """
        Select a random personality profile from available profiles.
        
        Returns:
            Tuple of (profile_name: str, profile_content: str)
        """
        name = np.random.choice(self.profile_names)
        return name, self.profiles[name]

    def generate_response(
        self, 
        user_response: str = "I don't have much to say.", 
        personality: str | None = None, 
        update_memory: bool = True, 
        N: int = 3, 
        example_joke: str | None = None
    ) -> list[str]:
        """
        Generate joke responses based on user input and selected personality.
        
        This method:
        1. Selects or uses specified personality profile
        2. Constructs system instruction with output format
        3. Calls LLM with user input and optional example
        4. Parses XML response into list of jokes
        5. Optionally updates conversation memory
        
        Args:
            user_response: The user's input text (default: generic response)
            personality: Specific personality to use (None = random selection)
            update_memory: Whether to update conversation memory after generation
            N: Number of joke variations to generate
            example_joke: Optional example of user's preferred joke style
            
        Returns:
            List of joke strings. Returns fallback messages on error.
        """
        # Select personality profile (random or specified)
        _, profile = self.get_random_profile()
        if personality is not None:
            profile = self.profiles.get(personality, profile)

        # Construct system instruction with formatted output specification
        system_instruction = f"{profile}\n{self.output_format.replace('[N]', str(N))}"

        # Build prompt with optional example joke
        prompt = user_response
        if example_joke:
            prompt = (
                f"{user_response}\n\n"
                f"<example_of_joke_user_liked>{example_joke}</example_of_joke_user_liked>"
            )

        # Call LLM
        model_response = self._call_llm(system_instruction, prompt)
        
        if not model_response:
            return ["I'm done for the day"]

        # Strip markdown code fences if present (e.g., ```xml ... ```)
        model_response = model_response.strip()
        if model_response.startswith("```"):
            lines = model_response.split("\n")
            lines = [l for l in lines[1:] if l.strip() != "```"]
            model_response = "\n".join(lines)

        # Parse XML response to extract jokes
        try:
            parsed_response = ET.fromstring(model_response)
            response_jokes = list(map(lambda x: x.text, parsed_response.findall("joke")))
            if response_jokes is None or response_jokes[0] is None:
                return ["I'm not feeling so funny today."]
            if update_memory:
                self.update_memory(user_response, response_jokes[0])
        except Exception as e:
            print("Actual model response: ", model_response)
            return ["I lost my train of thought. Could you repeat that?"]

        return response_jokes

    def update_memory(self, user_response: str, model_response: str) -> str:
        """
        Update conversation memory with a concise summary of the exchange.
        
        Uses the LLM to generate a summary that captures key information
        from the conversation history, new user input, and agent response.
        
        Args:
            user_response: The user's most recent message
            model_response: The agent's most recent response
            
        Returns:
            Updated memory summary string
        """
        prompt = (
            f"previous summary: {self.memory}, "
            f"new user msg: {user_response}, "
            f"new system msg: {model_response}"
        )

        summary = self._call_llm(self.MEMORY_SYSTEM_INSTRUCTION, prompt)

        if summary:
            self.memory = summary
            print("New memory:\n", self.memory)

        return self.memory


class LLMAPIJokerAgent(LLMJokerAgent):
    """
    Joker agent implementation using the Google Gemini API.
    
    This implementation communicates with Google's Gemini models via the
    official Python SDK for reliable, production-ready joke generation.
    
    Attributes:
        client: Google GenAI API client instance
        model: Name of the Gemini model to use
    """
    
    def __init__(
        self, 
        prompts_dir: Path, 
        api_key: str | None = None, 
        model: str = "gemini-2.5-flash"
    ):
        """
        Initialize API-based joker agent with Google Gemini.
        
        Args:
            prompts_dir: Path to personality profile directory
            api_key: Google Cloud API key for Gemini access (reads from GEMINI_API_KEY env var if not provided)
            model: Gemini model identifier (default: gemini-2.5-flash)
        """
        super().__init__(prompts_dir)
        
        # Get API key from parameter or environment variable
        if api_key is None:
            api_key = os.environ.get('GEMINI_API_KEY')
            if not api_key:
                raise ValueError(
                    "No API key provided. Either pass api_key parameter or set GEMINI_API_KEY environment variable.\n"
                    "Get a new key at: https://aistudio.google.com/apikey"
                )
        
        self.client = genai.Client(api_key=api_key)
        self.model = model

    def _call_llm(self, system_instruction: str, user_content: str) -> str | None:
        """
        Call Gemini API to generate content.
        
        Args:
            system_instruction: System prompt defining agent behavior
            user_content: User's input message
            
        Returns:
            Generated text response, or None on failure
        """
        response = self.client.models.generate_content(
            model=self.model,
            config=types.GenerateContentConfig(
                system_instruction=system_instruction
            ),
            contents=user_content,
        )
        return response.text


class LLMCLIJokerAgent(LLMJokerAgent):
    """
    Joker agent implementation using the Gemini command-line interface.
    
    This implementation invokes the `gemini` CLI tool via subprocess,
    useful for testing or environments where direct API access is limited.
    
    Note: Requires the `gemini` CLI tool to be installed and accessible in PATH.
    """

    def __init__(self, prompts_dir: Path):
        """
        Initialize CLI-based joker agent.
        
        Args:
            prompts_dir: Path to personality profile directory
        """
        super().__init__(prompts_dir)

    def _call_llm(self, system_instruction: str, user_content: str) -> str | None:
        """
        Call Gemini via command-line interface.
        
        Executes the `gemini` CLI tool with a 60-second timeout and
        captures the response from stdout.
        
        Args:
            system_instruction: System prompt defining agent behavior
            user_content: User's input message
            
        Returns:
            Generated text response, or None on failure/timeout
        """
        try:
            result = subprocess.run(
                ["gemini", "-p", f"{system_instruction}\n\nUser: {user_content}"],
                capture_output=True,
                text=True,
                timeout=60
            )

            if result.returncode != 0:
                print(f"CLI Error: {result.stderr}")
                return None

            return result.stdout.strip() or None

        except subprocess.TimeoutExpired:
            print("CLI timeout")
            return None
        except FileNotFoundError:
            print("Gemini CLI not found.")
            return None


class MemoryAgent:
    """
    Linguistic memory system for tracking conversation context.
    
    This agent extracts and maintains semantic features from user messages,
    including subjects, objects, entities, verbs, and adjectives. It uses
    pronoun resolution to link references across messages and generates
    summaries that include both relevant and contrasting context.
    
    The memory system helps the joke agent understand conversation flow
    and generate contextually appropriate humor.
    
    Attributes:
        PRONOUNS: Set of common pronouns for reference resolution
        memory: List of extracted feature dictionaries
        max_memory_length: Maximum number of messages to retain
    """
    
    PRONOUNS = {
        "it", "they", "he", "she", "this", "that", "these", "those", 
        "i", "me", "my", "we", "us", "our", "you", "your"
    }

    def __init__(self, max_memory_length: int):
        """
        Initialize the memory agent.
        
        Args:
            max_memory_length: Maximum number of conversation turns to remember
        """
        self.memory = []
        self.max_memory_length = max_memory_length
        
    def user_update(self, content: str):
        """
        Process and store linguistic features from user input.
        
        Extracts entities, subjects, objects, verbs, and adjectives using
        spaCy NLP. If pronouns are detected in subjects, resolves them
        against the previous message's references.
        
        Args:
            content: User's message text to process and store
        """
        features = self._extract_features(content)
        
        # Pronoun resolution: link pronouns to previous message references
        if self.memory and any(s.lower() in self.PRONOUNS for s in features["subj"]):
            prev = self.memory[-1]
            features["resolved_refs"] = (
                prev.get("subj", []) + 
                prev.get("obj", []) + 
                prev.get("entities", [])
            )
        
        self.memory.append(features)
        
        # Maintain sliding window of recent messages
        if len(self.memory) > self.max_memory_length:
            self.memory = self.memory[-self.max_memory_length:]        

    def get_full_memory_summary(self, n_contrast: int = 1) -> str:
        """
        Generate a comprehensive memory summary with relevant and contrasting context.
        
        Creates a summary that includes:
        1. Recent relevant messages (connected by shared references)
        2. Contrasting context groups (unrelated conversation threads)
        
        This provides the joke agent with rich context while highlighting
        topic shifts that might be good fodder for humor.
        
        Args:
            n_contrast: Number of contrasting context groups to include
            
        Returns:
            Summary string with relevant and contrasting contexts separated by " | "
        """
        if not self.memory:
            return ""

        # Find messages relevant to the most recent message
        current_idx = len(self.memory) - 1
        relevant_idx = self._get_relevant_indices(current_idx)
        non_relevant_idx = [i for i in range(current_idx + 1) if i not in relevant_idx]

        # Group non-relevant messages into contrast clusters
        contrast_groups = []
        used = set()
        for idx in sorted(non_relevant_idx, reverse=True):
            if idx in used:
                continue
            group = [i for i in self._get_relevant_indices(idx) if i in non_relevant_idx]
            if group:
                contrast_groups.append(group)
                used.update(group)

        # Sample contrast groups for inclusion
        selected_contrasts = random.sample(
            contrast_groups, 
            min(n_contrast, len(contrast_groups))
        )

        # Build summary parts
        parts = [self._summarize([self.memory[i] for i in relevant_idx])]
        for group in selected_contrasts:
            parts.append(self._summarize([self.memory[i] for i in group]))

        return " | ".join(filter(None, parts))

    def _get_relevant_indices(self, target_idx: int) -> set[int]:
        """
        Find all memory indices relevant to the target message.
        
        Messages are considered relevant if they share non-pronoun
        references (entities, subjects, objects) with the target.
        
        Args:
            target_idx: Index of the target message in memory
            
        Returns:
            Set of indices for relevant messages (including target)
        """
        if target_idx < 0 or target_idx >= len(self.memory):
            return set()
        target_refs = self._get_refs(self.memory[target_idx])
        return {
            i for i in range(target_idx + 1) 
            if not target_refs.isdisjoint(self._get_refs(self.memory[i]))
        }

    def _get_refs(self, f: dict) -> set:
        """
        Extract all non-pronoun references from a feature dictionary.
        
        Args:
            f: Feature dictionary with subj, obj, entities, resolved_refs
            
        Returns:
            Set of reference strings (excluding pronouns)
        """
        all_refs = (
            f.get("subj", []) + 
            f.get("obj", []) + 
            f.get("entities", []) + 
            f.get("resolved_refs", [])
        )
        return {r for r in all_refs if r.lower() not in self.PRONOUNS}

    def _summarize(self, memory_list: list[dict]) -> str:
        """
        Create a compact text summary from a list of memory features.
        
        Combines linguistic features across messages into a single
        space-separated string, filtering out pronouns.
        
        Args:
            memory_list: List of feature dictionaries to summarize
            
        Returns:
            Compact summary string
        """
        if not memory_list:
            return ""
        
        tuples = []
        for f in memory_list:
            tuples.append((
                " ".join(w for w in f.get("subj", []) if w.lower() not in self.PRONOUNS),
                " ".join(f.get("verbs", [])),
                " ".join(f.get("adjectives", [])),
                " ".join(w for w in f.get("obj", []) if w.lower() not in self.PRONOUNS),
                " ".join(f.get("entities", [])),
            ))
        
        return " ".join(filter(None, (" ".join(filter(None, t)) for t in zip(*tuples))))

    def _extract_features(self, sentence: str) -> dict:
        """
        Extract linguistic features from a sentence using spaCy.
        
        Features extracted:
        - entities: Named entities (people, places, organizations, etc.)
        - subj: Subject tokens (nsubj, nsubjpass dependencies)
        - obj: Object tokens (dobj, pobj, iobj dependencies)
        - adjectives: Adjective tokens
        - verbs: Verb lemmas (base forms)
        - past_tense: Boolean indicating past tense usage
        - negated: Boolean indicating negation presence
        - numbers: Numeric expressions
        
        Args:
            sentence: Input text to analyze
            
        Returns:
            Dictionary of extracted features
        """
        doc = nlp(sentence)
        return {
            "entities": [e.text for e in doc.ents],
            "subj": [t.text for t in doc if "subj" in t.dep_],
            "obj": [t.text for t in doc if "obj" in t.dep_],
            "adjectives": [t.text for t in doc if t.pos_ == "ADJ"],
            "verbs": [t.lemma_ for t in doc if t.pos_ == "VERB"],
            "past_tense": any(t.tag_ == "VBD" for t in doc),
            "negated": any(t.dep_ == "neg" for t in doc),
            "numbers": [t.text for t in doc if t.like_num],
        }


class UserFeedbackTrackerAgent:
    """
    Agent for tracking user feedback on jokes by personality type.
    
    This agent implements a simple feedback collection system that stores
    positive and negative reactions to jokes, organized by personality.
    The feedback is used by the UCB (Upper Confidence Bound) algorithm
    to optimize personality selection over time.
    
    Attributes:
        positive_feedbacks: Dict mapping personalities to liked jokes
        negative_feedbacks: Dict mapping personalities to disliked jokes
        awaiting_feedback: Tuple of (personality, joke) pending user reaction
    """
    
    def __init__(self) -> None:
        """Initialize the feedback tracker with empty feedback collections."""
        self.positive_feedbacks = defaultdict(list)
        self.negative_feedbacks = defaultdict(list)
        self.awaiting_feedback = None
        
    def await_feedback(self, joke: str, personality: str):
        """
        Register a joke awaiting user feedback.
        
        Should be called immediately after presenting a joke to the user.
        
        Args:
            joke: The joke text that was presented
            personality: The personality profile that generated the joke
        """
        self.awaiting_feedback = (personality, joke)
        
    def process_feedback(self, polarity: bool):
        """
        Record user feedback for the awaiting joke.
        
        Adds the joke to either positive or negative feedback lists
        based on the user's reaction.
        
        Args:
            polarity: True for positive feedback (like), False for negative (dislike)
        """
        if self.awaiting_feedback is None:
            return
        
        feedbacks = self.positive_feedbacks if polarity else self.negative_feedbacks
        feedbacks[self.awaiting_feedback[0]].append(self.awaiting_feedback[1])

In [ ]:
# ============================================================================
# Agent Initialization (Multi-Language)
# ============================================================================

# Load spaCy model based on configured language
nlp = spacy.load(current_lang['spacy_model'])
print(f"✓ Loaded spaCy model: {current_lang['spacy_model']}")

# Initialize CLI-based joker agent (uses gemini command-line tool)
joker_agent_cli = LLMCLIJokerAgent(os.getcwd()/Path("prompts"))

# Initialize API-based joker agent (uses Google Gemini API)
joker_agent = LLMAPIJokerAgent(os.getcwd()/Path("prompts"))

# Initialize memory agent with 10-message sliding window
memory = MemoryAgent(10)

# Initialize feedback tracking agent
feedback = UserFeedbackTrackerAgent()

# Initialize speech recognition engine
recognizer = sr.Recognizer()

<div style="text-align: center !important; max-width: 600px; margin: 0 auto;">

# JokeBloke

**🇳🇱 Nederlands:**  
Druk op `boot recorder` om hem wakker te maken. Druk vervolgens op `record` om tegen hem te praten. Als je klaar bent met praten, druk dan op `stop` en wacht geduldig op zijn reactie.

**🇩🇪 Deutsch:**  
Drücken Sie `boot recorder`, um ihn aufzuwecken. Drücken Sie dann `record`, um mit ihm zu sprechen. Wenn Sie fertig sind, drücken Sie `stop` und warten Sie geduldig auf seine Antwort.

**🇬🇧 English (UK):**  
Press `boot recorder` to wake him up. Then press `record` to talk to him. When you're done talking, press `stop` and wait patiently for his response.

**🇮🇹 Italiano:**  
Premi `boot recorder` per svegliarlo. Poi premi `record` per parlargli. Quando hai finito di parlare, premi `stop` e aspetta pazientemente la sua risposta.

---

*Taal wijzigen / Sprache ändern / Change language / Cambiare lingua: Pas de `LANGUAGE` variabele aan in cel 2 ('nl', 'de', 'en-gb', of 'it')*

</div>

In [ ]:
# ============================================================================
# Audio Processing and UI Helper Functions
# ============================================================================

def get_audio_as_wav_bytes(recorder):
    """
    data = recorder.audiodata
    srate = recorder.sampleRate
    
    if data is None or len(data) == 0:
        raise ValueError("No audio recorded!")

    # Convert float32 audio to int16 format for WAV
    data_int16 = (data * 32767).astype(np.int16)

    # Create WAV file in memory
    wav_bytes = BytesIO()
    with wave.open(wav_bytes, 'wb') as wf:
        wf.setnchannels(1)  # Mono audio
        wf.setsampwidth(2)  # 16-bit samples
        wf.setframerate(srate)
        wf.writeframes(data_int16.tobytes())
    wav_bytes.seek(0)
    return wav_bytes


def transcribe(recorder):
    """
    Transcribe recorded audio to text using Google Speech Recognition.
    
    Converts the recorder's audio to WAV format and sends it to Google's
    speech recognition API for transcription.
    
    Args:
        recorder: ipyaudio.AudioRecorder instance with recorded audio
        
    Returns:
        str: Transcribed text, or None if recognition fails
    """
    wav_bytes = get_audio_as_wav_bytes(recorder)

    with sr.AudioFile(wav_bytes) as source:
        audio = recognizer.record(source)
    
    try:
        text = recognizer.recognize_google(audio, language='nl-NL')
        print("Transcriptie:", text)
        return text
    except sr.UnknownValueError:
        print("Kon audio niet verstaan")
    except sr.RequestError as e:
        print(f"API fout: {e}")
    return None


def speak_text(text, save_file=None):
    """
    Convert text to speech and play it using Google TTS and pygame.
    
    Generates an MP3 file using Google Text-to-Speech, then plays it
    through pygame's mixer. Optionally saves the MP3 file to disk.
    In environments without audio devices (like containers), prints the text instead.
    
    Args:
        text: The text to convert to speech and play
        save_file: Optional path to save the MP3 file (e.g., "output.mp3")
    
    Returns:
        str: Path to saved MP3 file if save_file is provided, None otherwise
    """
    Convert text to speech and play it using Google TTS and pygame.
    
    Generates an MP3 file using Google Text-to-Speech, then plays it
    through pygame's mixer. This function blocks until playback completes.
    In environments without audio devices (like containers), prints the text instead.
    
    Args:
        text: The text to convert to speech and play
    """
    try:
        # Create MP3 file (temporary or saved)
        if save_file:
            mp3_path = save_file
            gTTS(text, lang='en', tld='co.uk').save(mp3_path)
        else:
            with tempfile.NamedTemporaryFile(suffix='.mp3', delete=False) as f:
                mp3_path = f.name
                gTTS(text, lang='en', tld='co.uk').save(f.name)
        
        # Play the audio
        # Configure SDL for headless environment
        os.environ.setdefault("SDL_AUDIODRIVER", "dummy")
        pygame.mixer.init()
        pygame.mixer.music.load(mp3_path)
        pygame.mixer.music.play()
            # Wait for playback to complete
        while pygame.mixer.music.get_busy():
            pygame.time.wait(100)
        
        return mp3_path if save_file else None
    except Exception as e:
        # No audio device available (common in containers/headless environments)
        print(f"🔊 JokeBloke says: {text}")


# ============================================================================
# Loading Messages for User Engagement During LLM Processing
# ============================================================================

LOADING_MESSAGES = [
    "Even geduld, ik werk eraan...",
    "Komedie kost tijd, anders dan het geduld van mijn ex...",
    "Nog aan het denken... in tegenstelling tot mijn comedycarrière gaat dit ergens heen",
    "Een momentje, genie aan het werk... of op z'n minst lichte entertainment",
    "Humor aan het laden... even geduld alstublieft",
    "Mijn schrijvers staken weer...",
    "Buffer humor... heb je geprobeerd me uit en aan te zetten?",
    "Raadpleeg mijn innerlijke clown...",
    "De clou zit vast in het verkeer...",
    "Bijna klaar... comedy goud groeit niet op bomen",
    "Even volhouden, ik ben grappiger dan deze pauze doet vermoeden",
    "Bezig met verwerken... deze grap kan maar beter goed zijn",
    "Mijn humorbot heeft even nodig...",
    "Grap aan het laden... in tegenstelling tot mijn liefdeleven komt dit wel af",
    "Ik stel niet uit, ik bouw spanning op...",
    "Rome is niet in één dag gebouwd, comedy goud ook niet",
]


def generate_with_loading_messages(
    generator_func, 
    on_message=None, 
    min_delay: float = 2.0, 
    max_delay: float = 5.0
):
    """
    Execute a long-running function while displaying rotating loading messages.
    
    Runs the generator function in a background thread and periodically calls
    the on_message callback with humorous loading messages to keep the user
    engaged during LLM processing.
    
    Args:
        generator_func: Callable that performs the long-running operation
        on_message: Optional callback function(message: str) for displaying messages
        min_delay: Minimum seconds between loading messages
        max_delay: Maximum seconds between loading messages
        
    Returns:
        The return value from generator_func
        
    Raises:
        Any exception raised by generator_func
    """
    result = []
    error = []
    done = threading.Event()

    def background_task():
        """Execute the generator function and capture result or error."""
        try:
            result.append(generator_func())
            
        return mp3_path if save_file else None
        except Exception as e:
            error.append(e)
        finally:
            done.set()

    # Start background thread for generation
    thread = threading.Thread(target=background_task, daemon=True)
    thread.start()

    # Display loading messages until generation completes
    used_messages = []
    while not done.is_set():
        delay = random.uniform(min_delay, max_delay)
        if done.wait(timeout=delay):
            break
        
        # Pick a message we haven't used yet to avoid repetition
        if len(used_messages) >= len(LOADING_MESSAGES):
            used_messages = []
        available = [m for m in LOADING_MESSAGES if m not in used_messages]
        message = random.choice(available)
        used_messages.append(message)
        
        if on_message:
            on_message(message)

    thread.join()

    # Propagate any errors from the background thread
    if error:
        raise error[0]
    return result[0]


def save_wav_from_recorder(recorder, filename: str = "recording.wav"):
    """
    Save recorded audio to a WAV file on disk.
    
    Utility function for debugging or archiving audio recordings.
    
    Args:
        recorder: ipyaudio.AudioRecorder instance with recorded audio
        filename: Output filename for the WAV file
        
    Returns:
        str: The filename if successful, None if no audio to save
    """
    data = recorder.audiodata
    sr = recorder.sampleRate
    
    if data is None or len(data) == 0:
        print("No audio data to save!")
        return None

    # Convert to 16-bit PCM
    data_int16 = (data * 32767).astype(np.int16)

    with wave.open(filename, "wb") as f:
        f.setnchannels(1)
        f.setsampwidth(2)
        f.setframerate(sr)
        f.writeframes(data_int16.tobytes())

    print(f"Saved: {filename}")
    return filename


def plot_audio(rec):
    """
    Visualize the recorded audio waveform using matplotlib.
    
    Useful for debugging audio quality and verifying recordings.
    
    Args:
        rec: ipyaudio.AudioRecorder instance with recorded audio
    """
    plt.figure(figsize=(10, 3))
    plt.plot(rec.audiodata)
    plt.title("Recorded Audio Waveform")
    plt.xlabel("Samples")
    plt.ylabel("Amplitude")
    plt.show()


# ============================================================================
# UI Widget Setup
# ============================================================================

# Audio recorder widget
rec = ipyaudio.AudioRecorder()

# Status output for recorder state messages
status_out = Output(layout={'padding': '5px'})

# Feedback UI components
text_label = widgets.Label(value="Vind je dit leuk?")
like_button = widgets.Button(description=": )", button_style='success')
dislike_button = widgets.Button(description=": (", button_style='danger')

# Loading indicator with animated GIF
loading_indicator = widgets.HTML(
    value='<img src="https://i.gifer.com/ZZ5H.gif" width="30" style="vertical-align:middle;"> '
          '<span>Bedenk iets grappigs...</span>'
)
loading_indicator.layout.display = 'none'

# Output area for feedback messages
output = widgets.Output()


def set_loading_state(is_loading: bool):
    """
    Enable or disable UI elements based on loading state.
    
    During joke generation:
    - Disables feedback buttons to prevent premature clicks
    - Makes the recorder appear disabled (CSS opacity/pointer-events)
    - Shows loading indicator, hides feedback prompt
    
    Args:
        is_loading: True to show loading state, False to show interactive state
    """
    like_button.disabled = is_loading
    dislike_button.disabled = is_loading
    
    # ipyaudioworklet doesn't have a disabled property, so we use CSS
    if is_loading:
        rec.layout.opacity = '0.5'
        rec.layout.pointer_events = 'none'
    else:
        rec.layout.opacity = '1'
        rec.layout.pointer_events = 'auto'
    
    loading_indicator.layout.display = 'inline' if is_loading else 'none'
    text_label.layout.display = 'none' if is_loading else 'inline'


# ============================================================================
# Event Handlers
# ============================================================================

@status_out.capture(clear_output=True)
def status_changed(change):
    """Display recorder status changes."""
    print("Status:", change.new)


@status_out.capture()
def on_status_change(change):
    """
    Main event handler for audio recorder status changes.
    
    Triggered when the user stops recording. This function orchestrates
    the entire joke generation pipeline:
    
    1. Transcribe audio to text
    2. Update conversation memory with user input
    3. Select optimal personality using UCB algorithm
    4. Generate joke(s) with loading messages
    5. Speak the joke back to the user
    6. Wait for user feedback
    
    The UCB (Upper Confidence Bound) algorithm balances exploration of new
    personalities with exploitation of personalities that have received
    positive feedback in the past.
    """
    if change.new in ("STOPPED", "RECORDED"):
        # Step 1: Transcribe user's voice input
        user_input = transcribe(rec)
        if not user_input:
            return

        # Step 2: Update conversation memory
        memory.user_update(user_input)

        personalities = joker_agent.profile_names

        # Step 3: Select personality using UCB (Upper Confidence Bound) algorithm
        # This balances exploration (trying untested personalities) with
        # exploitation (using personalities that have received positive feedback)
        
        # Calculate total feedback count across all personalities
        n_total = sum(
            len(feedback.positive_feedbacks[pers]) + len(feedback.negative_feedbacks[pers])
            for pers in personalities
        )

        ucb_scores = []
        for pers in personalities:
            positive = len(feedback.positive_feedbacks[pers])
            negative = len(feedback.negative_feedbacks[pers])
            n = positive + negative

            if n == 0:
                # Untested personalities get infinite UCB to encourage exploration
                ucb_scores.append(float('inf'))
            else:
                # UCB score = mean success rate + exploration bonus
                mean = positive / n
                exploration_bonus = np.sqrt(2 * np.log(n_total + 1) / n)
                ucb_scores.append(mean + exploration_bonus)

        # Pick personality with highest UCB score (random tiebreak for infinite scores)
        max_ucb = max(ucb_scores)
        best_indices = [i for i, score in enumerate(ucb_scores) if score == max_ucb]
        personality_pick = np.random.choice(best_indices)
        personality = personalities[personality_pick]
        
        print("Persoonlijkheid:", personality)

        # Step 4: Get most recent liked joke for this personality as an example
        liked_jokes = feedback.positive_feedbacks[personality]
        example_joke = liked_jokes[-1] if liked_jokes else None

        def update_loading_message(msg):
            """Callback to update the loading indicator with a new message."""
            loading_indicator.value = (
                f'<img src="https://i.gifer.com/ZZ5H.gif" width="30" '
                f'style="vertical-align:middle;"> <span>{msg}</span>'
            )

        # Step 5: Generate joke with loading messages
        set_loading_state(True)
        try:
            # Build prompt with conversation history and current input
            prompt = f"<history>{memory.get_full_memory_summary(2)}</history>\n{user_input}"
            
            jokes = generate_with_loading_messages(
                lambda: joker_agent.generate_response(
                    prompt, 
                    update_memory=False, 
                    personality=personality, 
                    N=1, 
                    example_joke=example_joke
                ),
                on_message=update_loading_message
            )
        finally:
            set_loading_state(False)

        print("Reactie:", jokes[0])

        # Step 6: Speak the joke and await feedback
        speak_text(jokes[0])
        feedback.await_feedback(jokes[0], personality)


def on_like_clicked(b):
    """
    Handle positive feedback (like button clicked).
    
    Records the positive feedback, speaks a thank you message,
    and displays confirmation to the user.
    """
    with output:
        clear_output()
        feedback.process_feedback(True)
        speak_text("Dank je!")
        print("Je vond dit leuk!")


def on_dislike_clicked(b):
    """
    Handle negative feedback (dislike button clicked).
    
    Records the negative feedback, speaks a self-deprecating response,
    and displays confirmation to the user.
    """
    with output:
        clear_output()
        feedback.process_feedback(False)
        speak_text("Wow, lastig publiek!")
        print("Je vond dit niet leuk!")


# ============================================================================
# Wire Event Handlers to Widgets
# ============================================================================

rec.observe(status_changed, "status")
rec.observe(on_status_change, "status")
like_button.on_click(on_like_clicked)
dislike_button.on_click(on_dislike_clicked)


# ============================================================================
# Display UI with Custom Styling
# ============================================================================

# Comprehensive CSS styling for the notebook interface
STYLE_SHEET_CONTENT = """
/* Center the layout and set a max-width */
.jp-Cell-outputArea {
    display: flex;
    flex-direction: column;
    align-items: center;
}

.jp-OutputArea-child {
    max-width: 600px;
    width: 100%;
    padding: 5px 0;
    box-sizing: border-box;
}

/* Hide the audio player widget */
.jupyter-widgets audio,
.jupyter-widgets video {
    display: none !important;
}

/* Style the buttons - auto height to fit content */
.widget-button {
    background-color: #f0f0f0;
    border: 1px solid #ccc;
    border-radius: 8px;
    padding: 4px 12px;
    margin: 5px;
    font-size: 14px;
    font-weight: bold;
    cursor: pointer;
    transition: background-color 0.3s, transform 0.1s;
    box-sizing: border-box;
    height: auto !important;
    min-height: unset !important;
    line-height: 1.4;
}

.widget-button:hover {
    background-color: #e0e0e0;
}

.widget-button:active {
    transform: scale(0.98);
}

.widget-button:disabled {
    background-color: #f5f5f5;
    color: #aaa;
    cursor: not-allowed;
}

.widget-button.mod-success {
    background-color: #28a745;
    color: white;
    border-color: #28a745;
}

.widget-button.mod-success:hover {
    background-color: #218838;
}

.widget-button.mod-danger {
    background-color: #dc3545;
    color: white;
    border-color: #dc3545;
}

.widget-button.mod-danger:hover {
    background-color: #c82333;
}

/* Style the output text area - auto-grow with content */
.widget-output {
    width: 100%;
    min-height: 50px;
    max-height: none;
    overflow: visible;
}

.lm-Widget {
    text-align: center;
}

.widget-output .jp-RenderedText {
    background-color: #f8f9fa;
    border: 1px solid #dee2e6;
    border-radius: 8px;
    padding: 15px;
    margin-top: 10px;
    text-align: left;
    min-height: 40px;
    height: auto;
}

.widget-output .jp-RenderedText pre {
    white-space: pre-wrap;
    word-wrap: break-word;
    font-family: monospace;
    font-size: 14px;
    margin: 0;
}

/* Style the text label */
.widget-label {
    font-weight: bold;
    font-size: 16px;
    text-align: center;
}

/* Style the loading indicator */
.widget-html-content {
    display: flex;
    align-items: center;
    gap: 8px;
}

.widget-html-content img {
    width: 30px;
    height: 30px;
}

.widget-html-content span {
    font-style: italic;
    color: #666;
}

/* Like/dislike buttons sizing - auto height */
.widget-button.mod-success,
.widget-button.mod-danger {
    min-width: 70px;
    font-size: 18px;
    padding: 6px 16px;
    height: auto !important;
    min-height: unset !important;
}
"""

# Inject CSS into notebook
display(widgets.HTML(f"<style>{STYLE_SHEET_CONTENT}</style>"))

# Display initial status message
status_out.append_stdout("Recorder klaar.\n")

# Assemble and display the feedback UI
feedback_ui = widgets.VBox([
    widgets.HBox(
        [text_label, loading_indicator], 
        layout=widgets.Layout(justify_content='center')
    ),
    widgets.HBox(
        [like_button, dislike_button], 
        layout=widgets.Layout(justify_content='center')
    ),
    output
], layout=widgets.Layout(align_items='center'))

# Display all UI components
display(rec)
display(status_out)
display(feedback_ui)

HTML(value='<style>\n/* Center the layout and set a max-width */\n.jp-Cell-outputArea {\n    display: flex;\n …

AudioRecorder()

Output(layout=Layout(padding='5px'), outputs=({'output_type': 'stream', 'name': 'stdout', 'text': 'Recorder re…